In [1]:
import numpy as np
import scipy as sp

In [2]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [3]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [4]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation
        # Refer to this source for derivation: https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()

### Finishing the controller
In this notebook, you'll take your previous implementation where we just controller altitude (z-position) and expand it to build the full position-attitude loop.

**Note that I updated the controller params class with some extra parameters!**

#### Recommended steps
- Copy your 3D quadrotor model (y)
- Copy your controller implementation (y)
- Expand the controller class ()
    - Compute the desired roll/pitch using the equations from the lecture ()
    - Compute the attitude errors ()
    - Compute the desired angular acceleration ()
    - Compute `u_2` ()
    - Tune your system (the gains provided _should_ provide a relatively stable starting point!) ()

### The dynamics

In [5]:
# 3D Quadrotor
class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:
        # NOTE(roy): I advise you to scroll through dynamics.py and see what the __init__
        # function in the base class does!
        super().__init__()

        #(faiza) __init__ of the dynamics base-class (assuming for class SimulatorBase) instantiates by dynamics, controller, trajectory params each inheriting from their respective base class
        # and sets the timestep (dt) for use in the simulation and for the dynamics class
        # and sets up initial state (of position & velocity)

    '''
    - Implement the mixing matrix (i.e. compute u1 and u2 from the rotor rates)
    - Modify the state_derivative functions to take a vector for u2 rather than a float (because now of course that's 3 numbers!)
    - Modify the accel formulation in your model to 3D (see equations in slides)
    - Modify the angular_accel formulation in your model to 3D (see equations in slides)
    '''
    
    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:
        # NOTE(roy): To see how complex this can get visit https://arxiv.org/pdf/1601.00733.pdf
        rates_squared = np.square(rotor_rates) 
        rotor_thrust_array = p.k_thrust * rates_squared #F = k_t * omega^2
            
        return (rotor_thrust_array)
        #this model has 4 rotors -- and can teeter on all 3 axis
    #def rotor_torque_model(rotor_rate: np.ndarray) -> np.ndarrary:

    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:
        # Get all 4 rotor rates
        F1, F2, F3, F4 = FullQuadrotorDynamics.rotor_thrust_model(input.rotor_rates)
        forces = np.array([[F1], [F2], [F3], [F4]]) #really just u1 components

        # NOTE(roy) Implement the thrust/torque calculation we discussed in the slides
        #u1 = F1 + F2 + F3 + F4 #thrust (summed forces)
        # OLD u2 = (F1 - F2) * (p.arm_length/2) #torque (--- and arm_length should be a global)

        #u2 would be an array of the moments of inertia of each of axes
        #mixing matrix contains info of u1 & u2, u2 has moments about each axis
        L = p.arm_length
        k = p.k_thrust/p.k_torque
        mixing_matrix = np.array([
                                  [1, 1, 1, 1],
                                  [0, L, 0, -1 * L],
                                  [-1 * L, 0 , L, 0],
                                  [k, -1 * k, k, -1 * k]
                                  ])

        u = mixing_matrix @ forces
        #print (u)

        u1 = u[0][0]
        u2 = u[1:4]
        #print (u1, u2)

        
        # NOTE(roy) this local function is a wrapper around the class state_derivative function
        # all this does is convert the our state dataclass from/into a vector that the scipy
        # integration solver can work with. See dynamics.py for the definition of these
        # from/to state vector functions.
        def state_derivative(t: float, state: np.ndarray) -> np.ndarray: #dstate is state deriv?
            return FullQuadrotorDynamics.state_derivative(
                QuadrotorState.from_state_vector(state),
                u1,
                u2,
            ).to_state_vector()

        state_vector = self.state.to_state_vector() #new state vector ready to use with scipy
        # raise NotImplementedError("Use the scipy function solve_ivp to integrate to the next timestep")

        #solution = {}  # NOTE(roy) solution = sp.integrate.solve_ivp(.... (0,self.dt)
        solution = sp.integrate.solve_ivp(state_derivative, (0, self.dt), state_vector)
        
        self.state = QuadrotorState.from_state_vector(solution['y'][:, -1]) #(faiza) [ : , -1] means last element in all rows
        return self.state 

    @staticmethod
    def state_derivative(state: QuadrotorState, u1: np.array, u2: np.array) -> QuadrotorState:
#get q dot and integrate forward
        RAB = state.orientation
        
        accel = (np.array([0, 0, p.mass * p.g *-1]) + (RAB * np.array([0, 0, u1])))/p.mass
        #print(accel.shape)

        #solving newton-euler for angular accel
        #M is array of momentums by each axis of rotation, torques
        M = np.transpose(u2)[0]
        #print("M", M)
        #print ("cross" , np.cross(state.angular_velocity, p.inertia))
        angular_accel = np.linalg.inv(p.inertia) @ (M - np.cross(state.angular_velocity, p.inertia @ state.angular_velocity)) #array of momentums 'M' are of u2 remember?

        #print(angular_accel.shape)
    
#omega ang vel
        # NOTE(roy) this might look a little confusing at first glance, but we're populating DERIVATIVES
        # of the state here! So position -> velocity, velocity -> acceleration, etc...
        # If you can think of a more readable way to do this (e.g. defining a QuadrotorStateDerivative class)
        # feel free to open a pull-request into the upstream repository!
        state_derivative = QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity=angular_accel,
        )
        
        return state_derivative

### The controller

In [6]:
@dataclass
class ControllerParams:
    """The same as above but for the controller gains"""
    # last controller params class with some extra parameters!

    # NOTE(roy) diag just means we get a matrix with these values on the diagonal,
    # so these are 3x3 matrices (print them if you're unsure!)
    # We use this formulation because it makes the math a bit more readable later on!
    K_p: np.ndarray = field(default_factory=lambda: np.diag([1, 1, 100]))
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0.5, 0.5, 10]))

    # Added gains for the attitude controller
    Ka_p: np.ndarray = field(default_factory=lambda: np.diag([300, 300, 50]))
    Ka_d: np.ndarray = field(default_factory=lambda: np.diag([50, 50, 20]))

    # Min/max rotor rates!
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()

In [23]:
# Controller should be edited from 4-Controller.ipynb to include next 2-axes
class Controller(ControllerBase):
    @staticmethod
    def rotor_rates_from_u(u: np.ndarray) -> np.ndarray:
        # NOTE(roy) implement your conversion from u (so that's [u1, u2_1, u2_2, u2_3] like we discussed, a 4-sized vector)
        # to rotor rates (which is also a four-sized vector)

        # NOTE(roy) the steps here are:
        #  1. convert u -> forces with the inverse mixing matrix
        #  2. convert forces -> rotor rates by inverting the rotor rate model
        #  3. limit the rotor rates with rotor_rate_min/rotor_rate_max above!
        #raise NotImplementedError("rotor_rates_from_u not implemented!")

        L = p.arm_length
        k = p.k_thrust/p.k_torque
        mixing_matrix = np.array([
                                  [1, 1, 1, 1],
                                  [0, L, 0, -1 * L],
                                  [-1 * L, 0 , L, 0],
                                  [k, -1 * k, k, -1 * k]
                                  ])
        #u is given as an arg to this func
        # 1: u to forces conversion
        force_array = np.linalg.inv(mixing_matrix) @ u

        #all undefined 'nans' should become zeros below
        for f in range(4):
            if force_array[f] < 0:
                force_array[f] = 0
        
        # 2: force array to the non-physical rotor rate array
        math_omegas = np.sqrt(force_array/p.k_thrust)

        #3: limit rotor rate elements to what is physically possible
        #print(math_omegas[1]) #returns nan (not a number) for index zero, may be undefined for negative nums
              
        for w in range(4):
            if math_omegas[w] < controller_p.rotor_rate_min:
                math_omegas[w] = controller_p.rotor_rate_min
            if math_omegas[w] > controller_p.rotor_rate_max:
                math_omegas[w] = controller_p.rotor_rate_max
                
        rotor_rates = math_omegas
        return (rotor_rates)
        #print(rotor_rates) 
        
    def step(self, t: float, trajectory: TrajectoryState, state: QuadrotorState) -> QuadrotorCommands:
        desired_position, desired_velocity = trajectory.position, trajectory.velocity
        actual_position, actual_velocity = state.position, state.velocity
        
        # NOTE(roy) first compute the desired acceleration using the equation from the lectures!
        # I already extracted the values you need to compute it above
        commanded_acceleration = controller_p.K_d @ (desired_velocity - actual_velocity) + controller_p.K_p @ (desired_position - actual_position)

        # NOTE(roy) now compute u_1 from the commanded acceleration using the linearized equation from the lectures!
        #u_1 = 0 ---- a single num
        u_1 = p.mass * (p.g + commanded_acceleration[2])
        #print(u_1)

        #- Compute the desired roll/pitch using the equations from the lecture (y)
        ## using a Rot3 method
        yaw, pitch, roll = state.orientation.to_yaw_pitch_roll() #returns an ndarray (see scrap)

        yaw_rots = np.array([
            [np.cos(yaw), np.sin(yaw)],
            [np.sin(yaw), -np.cos(yaw)]
        ])

        #roll_tobe, pitch_tobe = yaw_rots.T @ commanded_acceleration[0:2]
        pitch_tobe, roll_tobe = yaw_rots.T @ commanded_acceleration[0:2]
        
        #- Compute the attitude errors (y)
        #yaw last element
        #rots_tobe = np.array([pitch_tobe, roll_tobe, trajectory.yaw])
        rots_tobe = np.array([roll_tobe, pitch_tobe, trajectory.yaw])
        #rots_ = np.array([pitch, roll, yaw])
        rots_ = np.array([roll, pitch, yaw])

        att_err = rots_tobe - rots_
        #issue here?
        #print(att_err)
        #att_err = np.mod(att_err, np.pi) incorrect
        att_err = ((att_err + np.pi) % (2 * np.pi)) - np.pi
        
        #- Compute the desired angular acceleration (y)
        omega_err = np.array([0,0,trajectory.yaw_rate]) - state.angular_velocity
        
        #- Compute `u_2` (y)
        u_2 = p.inertia @ (controller_p.Ka_p @ att_err + controller_p.Ka_d @ omega_err) 
        
        u = np.array([u_1, *u_2])

        rotor_rates = Controller.rotor_rates_from_u(u)
        return QuadrotorCommands(rotor_rates)

In [24]:
#scrap
'''a = np.array([1,2,3])
print(type(a))
b,c,d = a
print ("b",b,"c",c, "d",d)
print(type(b))
s,d,f = np.array([4,6,8])
print ("s",s,"d", d, "f", f)'''

'a = np.array([1,2,3])\nprint(type(a))\nb,c,d = a\nprint ("b",b,"c",c, "d",d)\nprint(type(b))\ns,d,f = np.array([4,6,8])\nprint ("s",s,"d", d, "f", f)'

### The trajectory
Just something simple, mess around with it!

In [25]:
class JumpTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x = 0 if t < 1.0 or t > 5.0 else 1.0
        y = 0 if t < 2.0 or t > 5.0 else 1.0
        position = np.array([x, y, 0])

        yaw_rate = 0.5
        yaw = yaw_rate * t
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=yaw_rate)

In [26]:
class JustYawTrajectory(TrajectoryBase):
    def eval(self, t: float):
        position = np.array([0, 0, 0])

        yaw_rate = 0.25
        yaw = yaw_rate * t
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=yaw_rate)

In [172]:
class SmoothTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x = np.sin(t)
        #x = 2 + np.cos(10 * t) + 2* np.sin(5 * t)
        y = np.cos(t)
        #if t<=3 or t>6:
            #z = .1
        #else: z = 0
        #z = np.cos(x) + np.sin(x/4)
        z = np.cos(x)
        position = np.array([x, y, z])
        return TrajectoryState(t, position=position)

In [173]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [174]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.005,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        trajectory=SmoothTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [175]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 30.0)

Output()

### Looking at the controller behavior
If all is well, the quadrotor should follow the trajectory!

Try some experiments:
- Can you make the controller more/less damped and faster/slower to respond?
- Can you write a trajectory that has feasible velocities/positions?
    - Hint: the trajectory that's currently there is infeasible, why?

### Commit your updated notebook to your fork!